<a href="https://colab.research.google.com/github/kiptuidenis/SKIES/blob/main/image_classification_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# Visualization tools
import torchvision
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt
import pandas as pd

Confirm Pytorch can recognize the GPU

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()
device

device(type='cuda')

Loading both the *train* and *valid* datasets for **MNIST**

In [11]:
train_set = torchvision.datasets.MNIST("./data/", train=True, download=True)
valid_set = torchvision.datasets.MNIST("./data", train=False, download=True)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.86MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 126kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.23MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.86MB/s]


**Explore the MNIST Data**

In [20]:
x_0, y_0 = train_set[0]

In [21]:
x_0

We can see that train_set contains PIL images and there corresponding labels stored in a tuple.

**PIL** is a Library that is used for opening, manipulating, and manipulating images.

In [15]:
type(x_0)

PIL.Image.Image

**Tensors**

A tensor is an n-dimensional array representing any number of dimensions.

PIL images need to be converted into Tensors so that they can be processed with neural networks.




```transforms.Compose ``` is a class in the ```torchvision.transforms``` module. It is used to chain multiple image transformations (Passed as a list) together, sequentially.

In [24]:
trans = transforms.Compose([transforms.ToTensor()])
x_0_tensor = trans(x_0)

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

PyTorch uses a ```C x H x W ``` convention, which means the first dimension is the color channel, second is height, and the third is width.

Since these images are black and white, there is only 1 color channel.

In [25]:
x_0_tensor.size()

torch.Size([1, 28, 28])

In [26]:
x_0_tensor

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

By default, tensors are usually processed by CPU. To move it to GPU we use ```.cuda``` method

In [27]:
x_0_tensor.device

device(type='cpu')

In [28]:
x_0_gpu = x_0_tensor.cuda()

In [30]:
x_0_gpu.device


device(type='cuda', index=0)

In [35]:
x_0_tensor.to(device)


tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

**Transforms**

There are multiple ways we can apply a list of transforms to a dataset. One way is to set it to a dataset's ```transform``` variable.

In [33]:
train_set.transform = trans
valid_set.transform = trans

Compose(    ToTensor())

**DataLoaders**

We could show our model their entire dataset at once, but this is computationally expensive. Furthermore using batches is more efficient. A ```batch_size``` of 32 or 64 is usually sufficient for most machine learning problems.

In [36]:
batch_size = 32

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size)

**Creating the model**

The most basic Neural Network contains 4 components:


1.   A *Flatten* used to convert an n-dimensional array into a vector
2.   An input layer
3.   A hidden layer
4.   An output Layer





In [37]:
layers = []

**Flattening the image**

In [39]:
layers = [
    nn.Flatten()
]
layers

[Flatten(start_dim=1, end_dim=-1)]

**The Input Layer**

Pytorch needs to know the number of inputs in order to create weights. We will use a *Linear layer* that is *densely connected*, This means that each neuron and weight will affect each neuron in the next layer.

In [40]:
input_size = 1 * 28 * 28

Choosing the number of neurons is what puts the "science" in "data science" as it is a matter of capturing the statistical complexity of the dataset. We will use 512 neurons in our case.

In [53]:
layers.append(nn.Linear(input_size, 512))

We will apply the ```relu``` activation function, which will help our network make more sophiscticated guesses about data than if it were required to make guesses based on some strictly linear function.

In [56]:
layers.append(nn.ReLU())

In [57]:
layers

[Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=784, out_features=512, bias=True),
 ReLU()]

**The hidden Layer**

Since each neuron in the previous layer (the input layer in this case) computes a single number, the number of inputs to the hidden layer = the number of neurons in the previous layer. We can then choose the number of neurons for the hidden layer. In this case we also use 512

In [58]:
layers.append(nn.Linear(512, 512))

In [60]:
layers.append(nn.ReLU())

In [61]:
layers

[Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=784, out_features=512, bias=True),
 ReLU(),
 Linear(in_features=512, out_features=512, bias=True),
 ReLU()]

**The Output Layer**

Since the network needs to make a guess about a single image belonging to 1 of 10 categories, there will be 10 outputs.

We will not assign the reLU function to the output layer. Instead we will apply a loss function

In [62]:
n_classes = 10
layers.append(nn.Linear(512, n_classes))

In [63]:
layers

[Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=784, out_features=512, bias=True),
 ReLU(),
 Linear(in_features=512, out_features=512, bias=True),
 ReLU(),
 Linear(in_features=512, out_features=10, bias=True)]

**Compiling the Model**

A Sequential model expects a sequence of arguments not a list, so we can use the ``` * ``` operator to unpack our list of layers into a sequence.

In [64]:
model = nn.Sequential(*layers)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU()
  (5): Linear(in_features=512, out_features=10, bias=True)
)

Move it to device to be processed on a GPU

In [68]:
model.to(device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU()
  (5): Linear(in_features=512, out_features=10, bias=True)
)

Check which device a model is on

In [72]:
next(model.parameters()).device

device(type='cuda', index=0)

**Training the Model**

Fitting the data to the model

**Loss function**

Loss functions measure how well a machine learning model performs by quantifying the difference between predicted and actual values.

We will use CrossEntropy loss function which is designed to grade if a model predicted the correct category from a group of categories

In [73]:
loss_function = nn.CrossEntropyLoss()

**Optimizer**

Are functions/algorithms that update the model's weights/parameters to minize loss function

Next, we select an optimizer for our model. If the loss_function provides a grade, the optimizer tells the model how to learn from this grade to do better next time

In [74]:
optimizer = Adam(model.parameters())

**Calculating Accuracy**

Number of correct classifications divided by total predictions made.